# Verify the ProprioTunedCNN sensorprocessing

Reload a pre-trained model specified by an experiment trained by the Train-ProprioTuned-CNN notebook.

This notebook runs a number of experiments that illustrate the performance. 

In [1]:
import sys
sys.path.append("..")

from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
# from torchvision import models, transforms
#import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from pprint import pprint

from sensorprocessing.sp_propriotuned_cnn import VGG19ProprioTunedRegression, VGG19ProprioTunedSensorProcessing, ResNetProprioTunedRegression, ResNetProprioTunedSensorProcessing
# from robot.al5d_position_controller import RobotPosition

from demonstration.demonstration import Demonstration
import sensorprocessing.sp_helper as sp_helper

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# The experiment/run we are going to run: the specified model will be created
experiment = "sensorprocessing_propriotuned_cnn"
run = "vgg19_128"
# run = "resnet50_128"
# run = "vgg19_256"
# run = "resnet50_256"

exp = Config().get_experiment(experiment, run)

***ExpRun**: Loading pointer config file:
	C:\Users\lboloni\.config\BerryPicker\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\settings-LotziYoga.yaml
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\sensorprocessing_propriotuned_cnn\vgg19_128_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: sensorprocessing_propriotuned_cnn/vgg19_128 successfully loaded


### Check the model for an encoding mode

Check whether we can load back the model. This loads back the model, not the sensor processing part. 

In [3]:
if exp['model'] == 'VGG19ProprioTunedRegression':
    model = VGG19ProprioTunedRegression(exp, device)
elif exp['model'] == 'ResNetProprioTunedRegression':
    model = ResNetProprioTunedRegression(exp, device)
else:
    raise Exception(f"Unknown model {exp['model']}")

if exp['loss'] == 'MSELoss':        
    criterion = nn.MSELoss()
elif exp['loss'] == 'L1Loss':
    criterion = nn.L1Loss()

modelfile = pathlib.Path(exp.data_dir(), 
                         exp["proprioception_mlp_model_file"])
print(modelfile)
assert modelfile.exists()
model.load_state_dict(torch.load(modelfile))

c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\sensorprocessing_propriotuned_cnn\vgg19_128\proprioception_mlp.pth


C:\Users\lboloni\AppData\Local\Temp\ipykernel_52716\2709452742.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(modelfile))


<All keys matched successfully>

In [6]:
#
#  FIXME: this does not work on the resnet maybe the image format is not good
#

# Run the loaded model on all the specified testing data

inputlist = []
targetlist = []

transform = sp_helper.get_transform_to_sp(exp)

with torch.no_grad():
    for val in exp["testing_data"]:
        run = val[0]
        demo_name = val[1]
        camera = val[2]
        exp_demo = Config().get_experiment("demonstration", run)
        demo = Demonstration(exp_demo, demo_name)
        for i in range(demo.metadata["maxsteps"]):
            sensor_readings, _ = demo.get_image(i, camera=camera, transform=transform, device=device)
            z = model.encode(sensor_readings)
            break

print(f"The shape of the output is: {z.shape}")
print(f"One example z looks like this:\n{z}")


***ExpRun**: Experiment default config C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\demonstration\_demonstration.yaml was empty, ok.
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\demonstration\freeform_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: demonstration/freeform successfully loaded
The shape of the output is: torch.Size([1, 128])
One example z looks like this:
tensor([[-11.2902,   7.0178, -12.0119,   6.3957, -12.0665, -17.2990,   1.5932,
         -13.7000,   3.7809,   5.2647,  -7.9615,   3.8480,  13.7341,   8.1710,
          -6.8268,  -9.5708,  -9.2781,  -2.5865,  -6.2096,  -6.9986,   7.0621,
           3.8940,  -1.7261,  -5.9583,   4.3704,  -7.7594,  -3.8514,  -9.9369,
          14.1173, -12.5146,   3.8911,  -6.5296,  -6.6962, -11.0779,   4.4012,
           5.7404,   2.7184,   2.2072,  -2.1389,  -6.6922,  -5.2830,  

### Show how to create and use a SensorProcessing object


In [7]:
pprint(exp)
# Create the sensor processing object

# exp = Config().get_experiment("sensorprocessing_propriotuned_cnn", run)

sp = None
if exp["model"] == "VGG19ProprioTunedRegression":
    sp = VGG19ProprioTunedSensorProcessing(exp, device)
if exp["model"] == "ResNetProprioTunedRegression":
    sp = ResNetProprioTunedSensorProcessing(exp, device)
if sp is None:
    raise Exception(f"Could not instantiate sp for {exp['model']}")

Experiment:
    batch_size: 32
    data_dir: c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\sensorprocessing_propriotuned_cnn\vgg19_128
    epochs: 10
    exp_run_sys_indep_file: C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\sensorprocessing_propriotuned_cnn\vgg19_128.yaml
    experiment_name: sensorprocessing_propriotuned_cnn
    freeze_feature_extractor: true
    image_size:
    - 256
    - 256
    latent_size: 128
    learning_rate: 0.001
    loss: MSELoss
    model: VGG19ProprioTunedRegression
    model_name: VGG-19
    name: vgg19-128
    output_size: 6
    proprioception_input_file: train_inputs.pt
    proprioception_mlp_model_file: proprioception_mlp.pth
    proprioception_target_file: train_targets.pt
    proprioception_test_input_file: test_inputs.pt
    proprioception_test_target_file: test_targets.pt
    proprioception_testing_task: proprioception-uncluttered
    proprioception_training_task: proprioception-cluttered
    run

c:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\sensorprocessing\..\sensorprocessing\sp_propriotuned_cnn.py:171: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sel

In [ ]:
with torch.no_grad():
    for val in exp["testing_data"]:
        run = val[0]
        demo_name = val[1]
        camera = val[2]
        exp_demo = Config().get_experiment("demonstration", run)
        demo = Demonstration(exp_demo, demo_name)
        for i in range(demo.metadata["maxsteps"]):
            sensor_readings, _ = demo.get_image(i, camera=camera, transform=transform, device=device)
            z = sp.process(sensor_readings)
            break


print(z)
print(z.shape)

***ExpRun**: Experiment default config C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\demonstration\_demonstration.yaml was empty, ok.
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\demonstration\freeform_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: demonstration/freeform successfully loaded
[-11.29016      7.0178137  -12.0119295    6.395692   -12.066526
 -17.298992     1.5931913  -13.70003      3.7808697    5.2646704
  -7.961477     3.8479617   13.734113     8.170959    -6.826784
  -9.57076     -9.278096    -2.5864713   -6.2096214   -6.998599
   7.0621266    3.8940277   -1.7260776   -5.9582744    4.3704357
  -7.759405    -3.8514352   -9.936905    14.117271   -12.51461
   3.8910668   -6.5295997   -6.6962056  -11.077883     4.40119
   5.740445     2.7183769    2.207162    -2.13887     -6.6922364
  -5.282955    15.663632     3.6